In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from catboost import CatBoostClassifier

In [3]:
from joblib import load
pipeline_rf = load('./models/randomforest_pipeline.pkl')

In [4]:
pipeline_rf.named_steps

{'preprocessing': ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
                                  ['Group', 'Sex', 'Arrival mode', 'Injury',
                                   'Mental']),
                                 ('std', StandardScaler(), ['Age', 'HR']),
                                 ('rob', RobustScaler(),
                                  ['NRS_pain', 'SBP', 'RR', 'BT'])]),
 'classifier': RandomForestClassifier(random_state=42)}

In [5]:
expected_columns = pipeline_rf.named_steps['preprocessing'].feature_names_in_
expected_columns

array(['Group', 'Sex', 'Age', 'Arrival mode', 'Injury', 'Mental',
       'NRS_pain', 'SBP', 'HR', 'RR', 'BT'], dtype=object)

In [6]:
model_cb = CatBoostClassifier()
model_cb.load_model('./models/catboost_model.cbm')

In [7]:
model_cb.feature_names_

['Group',
 'Sex',
 'Age',
 'Arrival mode',
 'Injury',
 'Mental',
 'NRS_pain',
 'SBP',
 'HR',
 'RR',
 'BT']

In [8]:
nueva_muestra = pd.DataFrame([{
    'Group': 2,        # Tipo de centro de atención: 2 = Regional (ED de 4to nivel, mayor capacidad)
    'Sex': 1,          # Sexo del paciente: 1 = Mujer
    'Age': 68,         # Edad del paciente en años
    'Arrival mode': 3, # Modo de llegada: 3 = Vehículo privado (sin traslado especializado)
    'Injury': 2,       # Lesión presente: 2 = Sí (llegó por una lesión)
    'Mental': 1,       # Estado mental: 1 = Alerta (nivel más alto de conciencia)
    'NRS_pain': 3,     # Escala Numérica del Dolor (NRS): 3 = Dolor leve (0-3 leve, 4-6 moderado, 7-10 severo)
    'SBP': 130,        # Presión Arterial Sistólica (SBP): 130 mmHg (ligeramente elevada)
    'HR': 82,          # Frecuencia Cardíaca (HR): 82 latidos por minuto (normal)
    'RR': 20,          # Frecuencia Respiratoria (RR): 20 respiraciones/min (en el límite superior normal)
    'BT': 36.4         # Temperatura corporal (BT): 36.4 °C (normal)
}])

nueva_muestra

,Group,Sex,Age,Arrival mode,Injury,Mental,NRS_pain,SBP,HR,RR,BT
0,2,1,68,3,2,1,3,130,82,20,36.4


In [20]:
nueva_muestra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Group         1 non-null      int64  
 1   Sex           1 non-null      int64  
 2   Age           1 non-null      int64  
 3   Arrival mode  1 non-null      int64  
 4   Injury        1 non-null      int64  
 5   Mental        1 non-null      int64  
 6   NRS_pain      1 non-null      int64  
 7   SBP           1 non-null      int64  
 8   HR            1 non-null      int64  
 9   RR            1 non-null      int64  
 10  BT            1 non-null      float64
dtypes: float64(1), int64(10)
memory usage: 220.0 bytes


In [9]:
pred_rf = pipeline_rf.predict(nueva_muestra)
pred_cb = model_cb.predict(nueva_muestra)

print("Predicción RandomForest:", pred_rf[0])
print("Predicción CatBoost:", pred_cb[0])

Predicción RandomForest: 0
Predicción CatBoost: 0


In [10]:
casos_criticos = pd.DataFrame([
    {
        'Group': 1,        # Tipo de centro: 1 = Local (ED de 3er nivel)
        'Sex': 1,          # Sexo: 1 = Mujer
        'Age': 25,         # Paciente joven
        'Arrival mode': 3, # Llegó por cuenta propia (vehículo privado)
        'Injury': 2,       # Sí llegó por lesión
        'Mental': 1,       # Estado de conciencia: Alerta
        'NRS_pain': 3,     # Dolor leve
        'SBP': 85,         # Presión arterial sistólica baja (hipotensión leve)
        'HR': 80,          # Frecuencia cardíaca normal
        'RR': 20,          # Frecuencia respiratoria en límite normal
        'BT': 36.9         # Temperatura normal-alta
    },
    {
        'Group': 2,        # Tipo de centro: 2 = Regional (ED de 4to nivel)
        'Sex': 1,          # Sexo: 1 = Mujer
        'Age': 73,         # Persona adulta mayor
        'Arrival mode': 2, # Llegó en ambulancia pública
        'Injury': 2,       # Sí llegó por lesión
        'Mental': 4,       # Inconsciente (sin respuesta)
        'NRS_pain': 8,     # Dolor severo
        'SBP': 92,         # Presión sistólica apenas estable
        'HR': 135,         # Taquicardia (frecuencia elevada)
        'RR': 30,          # Frecuencia respiratoria elevada (disnea)
        'BT': 39.5         # Fiebre alta
    }
])


In [11]:
pred_rf = pipeline_rf.predict(casos_criticos)
pred_cb = model_cb.predict(casos_criticos)

print("RandomForest:", pred_rf)
print("CatBoost:", pred_cb)

RandomForest: [0 1]
CatBoost: [0 1]


In [12]:
casos_criticos_2 = pd.DataFrame([
    {
        'Group': 1,        # Tipo de centro: 1 = Local (ED de 3er nivel, menor capacidad)
        'Sex': 2,          # Sexo: 2 = Hombre
        'Age': 87,         # Edad avanzada, adulto mayor frágil
        'Arrival mode': 1, # Llegó caminando (sin traslado especializado)
        'Injury': 2,       # Lesión presente
        'Mental': 3,       # Responde solo al dolor (disminución del estado de conciencia)
        'NRS_pain': 9,     # Dolor severo
        'SBP': 85,         # Presión arterial baja (hipotensión)
        'HR': 120,         # Taquicardia (alta frecuencia cardíaca)
        'RR': 28,          # Frecuencia respiratoria elevada (signo de insuficiencia respiratoria)
        'BT': 39.2         # Fiebre alta (posible infección o sepsis)
    },
    {
        'Group': 2,        # Centro regional (ED 4to nivel, mayor capacidad)
        'Sex': 1,          # Mujer
        'Age': 73,         # Persona adulta mayor
        'Arrival mode': 2, # Ambulancia pública (traslado con alerta)
        'Injury': 2,       # Lesión presente
        'Mental': 4,       # Inconsciente (sin respuesta)
        'NRS_pain': 8,     # Dolor severo
        'SBP': 92,         # Presión sistólica límite bajo
        'HR': 135,         # Taquicardia severa
        'RR': 30,          # Disnea intensa
        'BT': 39.5         # Fiebre muy alta
    }
])

In [13]:
pred_rf_2 = pipeline_rf.predict(casos_criticos_2)
pred_cb_2 = model_cb.predict(casos_criticos_2)

print("RandomForest:", pred_rf_2)
print("CatBoost:", pred_cb_2)

RandomForest: [1 1]
CatBoost: [1 1]


In [14]:
muestra_3 = pd.DataFrame([
    {
        'Group': 1,        # Centro local (ED de 3er nivel, menor capacidad resolutiva)
        'Sex': 2,          # Hombre
        'Age': 87,         # Edad avanzada (adulto mayor, alto riesgo)
        'Arrival mode': 1, # Llegó caminando (sin asistencia médica previa)
        'BT': 39.2,        # Temperatura corporal elevada (fiebre alta → posible infección o sepsis)
        'NRS_pain': 9,     # Dolor severo (escala del 0 al 10 → ≥7 es crítico)
        'Injury': 2,       # Lesión presente (requiere evaluación rápida)
        'Mental': 3,       # Estado de conciencia alterado → responde solo al dolor
        'SBP': 85,         # Presión arterial sistólica baja (hipotensión: riesgo de shock)
        'HR': 120,         # Taquicardia (frecuencia cardíaca elevada)
        'RR': 28           # Frecuencia respiratoria elevada (signo de disnea o compensación)
    }
])

In [15]:
pred_rf_3 = pipeline_rf.predict(muestra_3)
pred_cb_3 = model_cb.predict(muestra_3)

print("RandomForest:", pred_rf_3)
print("CatBoost:", pred_cb_3)

RandomForest: [1]
CatBoost: [1]


Se han entrenado y validado dos modelos de clasificación binaria para predecir la condición de emergencia médica a partir de signos vitales, variables de llegada y estado mental del paciente.

- **RandomForest** y **CatBoost** mostraron el mejor rendimiento en precisión y recall.
- Ambos modelos predicen correctamente casos críticos simulados.
- El pipeline de RandomForest incluye codificación y escalado automático.
- CatBoost opera sobre datos crudos, facilitando su integración.